<div class='bar_title'></div>

*Simulation for Decision Making (S4DM)*

# Assignment 6: Output Analysis (Single Model)

Summer Semester 24


Gunther Gust & Ignacio Ubeda <br>
Chair for Enterprise AI <br>
Data Driven Decisions Group <br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Agenda

* Model Validation
* Output Analysis (Single Model)

## Car wash example

In [ ]:
import simpy
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
class EventLogger:
    def __init__(self):
        self.logs = []
        self.replication = None #replication number
        self.seed = None #seed used for the replication
    
    def set_replication_info(self, replication, seed):
        self.replication = replication
        self.seed = seed

    def log_car_arrival(self, entity, time):
        self.logs.append({'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_arrival', 'event_key': entity #simulation info
                          })
    
    def log_car_wash_request(self, entity, time):
        self.logs.append({'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_wash_request', 'event_key': entity #simulation info
                          })

    def log_car_departure(self, entity, time):
        self.logs.append({'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_departure', 'event_key': entity #simulation info
                          })

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)

In [ ]:
class Carwash:
    def __init__(self, env, num_machines, logger):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.logger = logger

    def wash(self):
        yield self.env.timeout(np.random.exponential(1/0.3))

In [ ]:
class Car:
    def __init__(self, env, name, carwash, logger):
        self.env = env
        self.name = name
        self.logger = logger

        self.env.process(self.run(carwash))

    def run(self, carwash):
        
        # Log the arrival
        self.logger.log_car_arrival(self.name, self.env.now)
        with carwash.machine.request() as request:
            yield request

            # Log the request for car wash
            self.logger.log_car_wash_request(self.name, self.env.now)
            yield self.env.process(carwash.wash())

            #Log the departure
            self.logger.log_car_departure(self.name, self.env.now)

In [ ]:
def car_generator(env, carwash, logger):
    car_count = 0

    # Create cars while the simulation is running
    while True:
        yield env.timeout(np.random.exponential(1/0.5))
        Car(env, f'Car {car_count}', carwash, logger)
        car_count += 1

## Run Simulation (now, for multiple replications)

In [ ]:
# parameters
NUM_MACHINES = 2    # Number of machines in the carwash
SIM_TIME = 8*60       # Simulation time in minutes
N_REPLICATIONS = 20 # Number of Replications

# Setup and start the simulation
print('Running Simulation...')

#define logger (same for all replications)
logger = EventLogger()

#Compute a pool of seeds that is larger than the number of replications
safe_factor = 10
pool_of_seeds = range(1, N_REPLICATIONS * safe_factor)

#get a list of seeds of length: N_REPLICATIONS from a pool of seeds. 
#We set replace=False to ensure that we don't reuse the same seed twice.
list_of_seeds = np.random.choice(pool_of_seeds, size=N_REPLICATIONS, replace=False)

for i, seed in enumerate(list_of_seeds):
    print(f'Running Replication {i} with seed: {seed} ...')

    #set random seed
    np.random.seed(seed)

    #set replication id and random seed
    logger.set_replication_info(i, seed)

    # Create an environment and start the setup process
    env = simpy.Environment()

    #define resources
    carwash = Carwash(env, NUM_MACHINES, logger)

    #define processes
    env.process(car_generator(env, carwash, logger))

    # Execute
    env.run(until=SIM_TIME)

print('... Done')

In [ ]:
events_df = logger.get_logs_df()

events_df

# Exercise / Tasks

**Tasks are independently of each other.**

## Task 1: Model Validation

We'd like to validate our model. We've measured the actual average waiting time (2.8 mins) and would like to compare it with the output of our simulation model. 

-----

**Task 1.1: Create a dataframe `results_df` that contains the waiting time for every entity (car) for every replication. Recall that the waiting time is the timespan between the "*car_wash_request*" and "*car_arrival*" event. For computing this you can choose whatever approach you want. Check Hint 1 for a suggestion.**

**Task 1.2: Once you have `results_df`, compute the average waiting time for every replication. For computing this you can choose whatever approach you want. Check Hint 2 for a suggestion.**

After Task 1.2 you should have a table that looks like this (not same values, but structure):

<img src="images/assignment6_validation_lecture.png" style="width:30%" />

Where you have an aggregated metric for every replication (in the lecture example, $Y_2$). In our case, the table should have (at least) 2 columns: a replication id and the average waiting time for that replication.

**Task 1.3: Compute a confidence interval for the average waiting time (accross replications). Use the following formula (Lecture 8, slide 30):**

<img src="images/assignment6_ciformula_lecture.png" style="width:20%" />

**For gettint the t value, you may want to use `stats.t.ppf` from `scipy`. Check the [reference](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html) for more details.**

**Task 1.4: You've measure the actual average wating time (2.8 mins). Create an hypothesis test, interpret it, and check if your model is valid. For computing this you can choose whatever approach you want. Check Hint 3 for a suggestion.**

-----

**Hint 1:** `events_df` is in [long format](https://www.statology.org/long-vs-wide-data/). To compute the waiting time would be easier if its wide. To reshape from long to wide you could use the function: [`pd.pivot_table`](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html)

**Hint 2:** For aggregating a value (column `v`) with a function `f`  for every group (column `g`) of a DataFrame `df`. You can use the following syntax: `df.groupby('g').agg({'v': [f]})`. Check the [reference](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html) for more details.

**Hint 3:** `scipy` has a working implementation of a t-test: `stats.ttest_1samp`. Check the [reference](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html) for more details.


In [ ]:
#long to wide format (Hint 1)
results_df = #your code here

#compute waiting time
results_df['waiting_time'] = #your code here

print(results_df.columns)

results_df

In [ ]:
#If you have followed Hint 1: Leave this code unchanged. If not, you can remove it.

#Note the "\"" character is used to split a long line of code into multiple lines
#We can therefore "chain" multiple operations together, which makes the code more readable

#1st: filter for replication 0
#2nd: sort by car arrival time (ascending)
#3rd: show the first 10 rows

results_df\
    .loc[results_df['replication_id'] == 0]\
    .sort_values(by='car_arrival', ascending=True)\
    .head(10)

In [ ]:
#Compute the average waiting time for every replication

#your code here

In [ ]:
#significance level
alpha = 0.05

#compute mean, std and sample size
sample_size = #your code here
sample_mean = #your code here
sample_std = #your code here

print(f"sample mean: {sample_mean}")
print(f"sample std: {sample_std}")

#Calculate t confidence interval

# Calculate the t-value for a 95% confidence interval (two-sided)
t_value = #your code here

# Calculate the half-width of the confidence interval
ci_halfwidth = #your code here

#calculate the lower and upper bounds of the confidence interval
ci_lb = #your code here
ci_ub = #your code here

print(f"t confidence int: [{ci_lb}, {ci_ub}]") 

In [ ]:
#perform  t-test
#your code here


#interpret the test
#your code here

Answer the following questions:

- Is the simulation model valid?

**Answer in this markdown chunk:**

- *Your answer here*

## Task 2: Number of Replications

We have run 100 replications of our simulation model and now we'd like to compute how many replications do we need to ensure a specific margin of error in our output measurement.

-----

**Task 2.1: With the available DataFrame (`replication_results`) compute the number of replications (R) needed to have an error $\epsilon = 0.2$ with a significance level $\alpha = 0.05.$ Use the following formula (Lecture 9, slide 27):**

<img src="images/assignment6_nreplications_lecture.png" style="width:20%" />

**For gettint the t value, you may want to use `stats.t.ppf` from `scipy`. Check the [reference](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html) for more details.**

**Task 2.2: Pack your implementation in a function `sample_size_calculator` and use it to answer how the sample size changes varying $\epsilon$ and $\alpha$**


In [ ]:
replication_results = pd.read_csv('replication_results.csv')

print(replication_results.shape)
replication_results

In [ ]:
epsilon = 0.2
alpha = 0.05

R0 = #your code here (initial number of replications)
t_value = #your code here
s0 = #your code here (initial standard deviation)

R = #your code here

R

Answer the following questions:

- How many replications are needed?

**Answer in this markdown chunk:**

- *Your answer here*

In [ ]:
def sample_size_calculator(alpha, epsilon, s0, R0):
    t_value = #your code here
    R = #your code here
    return R

In [ ]:
#Leave this code unchanged

#define epsilon and alpha ranges
epsilon_values = np.arange(0.0, 1.0, 0.05)
alpha_values = np.arange(0.0, 1.0, 0.05)

data = []

#iterate over epsilon and alpha values
for epsilon in epsilon_values:
    for alpha in alpha_values:
        
        #compute sample size
        try:
            R = sample_size_calculator(alpha, epsilon, s0, R0)
        except OverflowError:
            R = np.nan

        #append results to data
        data.append({'alpha': round(alpha, 2), 'epsilon': round(epsilon, 2), 'sample_size': R})
        
#convert data to DataFrame and pivot
df = pd.DataFrame(data)
df = df.pivot(index="alpha", columns="epsilon", values="sample_size")

df

In [ ]:
#Leave this code unchanged
sns.heatmap(df)

Answer the following questions:

1. How does the number of replications (R) change when changing $\epsilon$?
1. How does the number of replications (R) change when changing $\alpha$?

**Answer in this markdown chunk:**

1. *Your answer here*
1. *Your answer here*

## Task 3: Quantile Confidence Interval

We'd like to compute a confidence interval for a a specific quantile of the average waiting time.

-----

**Task 3.1: Sort the the available DataFrame (`replication_results`) by the `waiting_time_mean` ascending. Create a `rank` column to get the position of the rows of the sorted DataFrame.**

**Task 3.2: Compute the quantile (lower and upper) bounds for the quantile $p=0.8$ with a significance level $\alpha = 0.05$. Use the following formula (Lecture 9, slide 29):**

<img src="images/assignment6_quantilebounds_lecture.png" style="width:20%" />

**For gettint the z value, you may want to use `stats.norm.ppf` from `scipy`. Check the [reference](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) for more details.**

**Task 3.3: Compute the point estimate and the bounds of the confidence interval with the indexes of the sorted DataFrame (i.e. the `rank` column). Check the example on Lecture 9, slide 30.**

**Task 3.4: Finally, we'll use a another approach using the `.quantile()` function. Check the [reference](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html) for more details and compare it with the result of Task 3.3**

In [ ]:
#read csv
replication_results = pd.read_csv('replication_results.csv')
print(replication_results.shape)

#show the first 5 rows
replication_results.head()

In [ ]:
#show last 5 rows
replication_results.tail()

In [ ]:
#plot histogram
sns.histplot(replication_results['waiting_time_mean'], kde=True)

In [ ]:
#sort by waiting time mean (ascending)
replication_results = #your code here

#create rank column
replication_results['rank'] = #your code here

replication_results

In [ ]:
alpha = 0.05
quantile = 0.8
sample_size = #your code here

#compute z-value
z_value = #your code here

#compute quantile lower and upper bounds
quantile_lb = #your code here
quantile_ub = #your code here

print(quantile_lb, quantile_ub)

In [ ]:
#compute the row index of the quantile (lb, ub, and center)
quantile_lb_index = #your code here
quantile_index = #your code here
quantile_ub_index = #your code here

#compute the point estimate
point_estimate = #your code here

#compute the confidence interval
ci_lb = #your code here
ci_ub = #your code here

print(f"The point estimate is: {point_estimate}")
print(f"The {100*(1-alpha)}% CI is [{ci_lb}, {ci_ub}]")

In [ ]:
#compute the point estimate (now with the quantile function)
point_estimate_quantfunc = #your code here

#compute the confidence interval (now with the quantile function)
ci_lb_quantfunc = #your code here
ci_ub_quantfunc = #your code here

print(f"The point estimate (quantile function) is: {point_estimate_quantfunc}")
print(f"The {100*(1-alpha)}% CI (quantile function) is [{ci_lb_quantfunc}, {ci_ub_quantfunc}]")

Answer the following questions:

- What is the difference in values of both approaches? which one is better to use in which scenario?

**Answer in this markdown chunk:**

- *Your answer here*


## Task 4: Replication Method

Assuming now that our carwash runs 24/7, we're interested in implementing the replication method and differentiating between the transient vs the steady-state phase. 

For this task we'll change the metric to the queue length and we'll monitor it periodically to ensure we have exactly the same sampling frequency among different replications.

-----

**Task 4.1: Create a new method in the `EventLogger_Task4` to log the queue length. You should log (at least) the replication id, the time and the value of the queue length.**

**Task 4.2: Create a new process in the `Carwash_Task4` to monitor the queue length every 5 minutes. Note that this is a "ficticious" process that does not interact with the entity. The solely purpose is to monitor (and log) our variable of interest (queue length in this case).**

**Task 4.3: With this new changes, run your simulation replications (we provide this code for you) and filter the `events_t4_df` to get only the events related to the queue length.**

**Task 4.4: Plot the queue length over time. Note that since we have multiple records for the same time step (because we have one for each replication). The plot will aggregate the `y` value with a confidence interval around the mean.**

**Task 4.5: Determine a `T1` (the point in time where you should "cut" between transient and steady-state phase)**

**Task 4.6: Finally, compute the average queue length with and without the transient phase and compare your results. Note that you'll have to compute first the average queue length within replications and then, the average of this value accross replications.** 

In [ ]:
class EventLogger_Task4:
    def __init__(self):
        self.logs = []
        self.replication = None #replication number
        self.seed = None #seed used for the replication
    
    def set_replication_info(self, replication, seed):
        self.replication = replication
        self.seed = seed

    def log_car_arrival(self, entity, time):
        self.logs.append({'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_arrival', 'event_key': entity #simulation info
                          })
    
    def log_car_wash_request(self, entity, time):
        self.logs.append({'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_wash_request', 'event_key': entity #simulation info
                          })

    def log_car_departure(self, entity, time):
        self.logs.append({'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_departure', 'event_key': entity #simulation info
                          })

    #add a new method to log the queue length    
    #your code here
    

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)

In [ ]:
class Carwash_Task4:
    def __init__(self, env, num_machines, logger):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.logger = logger

        self.env.process(self.monitor_queue_length()) #NEW - start the monitor process

    def wash(self):
        yield self.env.timeout(np.random.exponential(1/0.3))
    
    #create a new process to monitor the queue length every 5 minutes
    #your code here

In [ ]:
#Leave this code unchanged

# parameters
NUM_MACHINES = 2       # Number of machines in the carwash
SIM_TIME = 7*24*60      # Simulation time in minutes
N_REPLICATIONS = 30     # Number of Replications

# Setup and start the simulation
print('Running Simulation...')

#define logger (same for all replications)
logger_t4 = EventLogger_Task4()

#Compute a pool of seeds that is larger than the number of replications
safe_factor = 10
pool_of_seeds = range(1, N_REPLICATIONS * safe_factor)

#get a list of seeds of length: N_REPLICATIONS from a pool of seeds. 
#We set replace=False to ensure that we don't reuse the same seed twice.
list_of_seeds = np.random.choice(pool_of_seeds, size=N_REPLICATIONS, replace=False)

for i, seed in enumerate(list_of_seeds):
    print(f'Running Replication {i} with seed: {seed} ...')

    #set random seed
    np.random.seed(seed)

    #set replication id and random seed
    logger_t4.set_replication_info(i, seed)

    # Create an environment and start the setup process
    env_t4 = simpy.Environment()

    #define resources
    carwash_t4 = Carwash_Task4(env_t4, NUM_MACHINES, logger_t4)

    #define processes
    #we use the same car_generator and Car class because has not changed
    env_t4.process(car_generator(env_t4, carwash_t4, logger_t4))

    # Execute
    env_t4.run(until=SIM_TIME)

print('... Done')

In [ ]:
#Leave this code unchanged
events_t4_df = logger_t4.get_logs_df()

print(events_t4_df.shape)
events_t4_df.head(20)

In [ ]:
#get only the queue_monitor events
queue_data = #your code here

In [ ]:
#set figure size
f, ax = plt.subplots(1, figsize=(20, 7))

#plot the queue length over time
sns.lineplot(data=#your code here
             x=#your code here
             y=#your code here 
             ax=ax)
plt.show()

In [ ]:
#Define the time point T1
T1 = #your code here

#set figure size
f, ax = plt.subplots(1, figsize=(20, 7))

#plot the queue length over time
sns.lineplot(data=#your code here
             x=#your code here
             y=#your code here
             ax=ax)
plt.axvline(x=T1, color='r', linestyle='--')
plt.show()

In [ ]:
#Check rule of thumb: TE > 10*T1
TE = (SIM_TIME-T1)
print(TE > 10*T1)

In [ ]:
#Compute the point estimate of the mean queue length within replications
#your code here

#Compute the point estimate of the mean queue length accross replications
point_estimator_with_transient = #your code here
point_estimator_with_transient

In [ ]:
#Compute the point estimate of the mean queue length within replications but deleting the first T1 minutes
#your code here

#Compute the point estimate of the mean queue length accross replications but deleting the first T1 minutes
point_estimator_without_transient = #your code here
point_estimator_without_transient

In [ ]:
#Compare the point estimates
point_estimator_with_transient - point_estimator_without_transient

Answer the following questions:

- What is the difference in the point estimates?

**Answer in this markdown chunk:**

- *Your answer here*